In [3]:
from __future__ import division
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt

In [4]:
ratings_train = pd.read_csv('./homework4/data/ratings_train.csv', header=None)
ratings_test = pd.read_csv('./homework4/data/ratings_test.csv', header=None)
ratings_fake = pd.read_csv('./homework4/data/ratings_fake.csv', header=None)

In [5]:
ratings_train.columns = ['user', 'movie', 'rating']
ratings_test.columns = ['user', 'movie', 'rating']
ratings_fake.columns = ['user', 'movie', 'rating']

In [6]:
A = ratings_fake.pivot(index='user', columns='movie').as_matrix()

In [205]:
mu = A.mean()
m = A.shape[0]
n = A.shape[1]
k = 5
T = 20
B = np.zeros(m)
C = np.zeros(n)
U = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (m))
V = np.random.multivariate_normal(np.zeros(k), (1/k) * np.identity(k), (n))

In [99]:
def compute_U(U, V, B, C, mu, A):
    """ Finds the value of U that maximizes
    the log-likehihoood. Inelegantly uses 
    the close form 
    """
    rights = []
    for i in range(U.shape[0]):
        js = []
        for j in range(V.shape[0]):
            r = (B[i] + C[j] + mu - A[i, j]) * V[j]
            js.append(r)
        rights.append(np.array(js).sum(axis=0))
    rights = np.array(right)
    
    lefts = []
    for j in range(V.shape[0]):
        arr = np.dot(V[j], V[j].T)
        lefts.append(arr)
    lefts = np.array(left)
    lefts = np.linalg.inv([[left.sum()]])
    
    return lefts * right

In [137]:
def compute_V(U, V, B, C, mu, A):
    """ Finds the value of U that maximizes
    the log-likehihoood. Inelegantly uses 
    the close form 
    """
    rights = []
    for j in range(V.shape[0]):
        js = []
        for i in range(U.shape[0]):
            r = (B[i] + C[j] + mu - A[i, j]) * U[j]
            js.append(r)
        rights.append(np.array(js).sum(axis=0))
    rights = np.array(right)
    
    lefts = []
    for i in range(U.shape[0]):
        arr = np.dot(U[i], U[i].T)
        lefts.append(arr)
    lefts = np.array(left)
    lefts = np.linalg.inv([[left.sum()]])
    
    return lefts * right

In [222]:
def update(U, V, B, C, mu, A):
    B = (-(U * V).sum(axis=1) - C - mu + A.sum(axis=1)) / n
    U = compute_U(U, V, B, C, mu, A)
    C = (-(U * V).sum(axis=1) - B - mu - A.sum(axis=1) / m)
    V = compute_V(U, V, B, C, mu, A)
    return -(np.dot(U, V.T) + B + C + mu - A)

In [186]:
# ((V * U) + B + C + mu - A)

In [168]:
lefts = []
for j in range(V.shape[0]):
    arr = np.dot(V[j], V[j].T)
    lefts.append(arr)
lefts = np.array(lefts)
lefts = np.linalg.inv([[lefts.sum()]])

In [170]:
right = []
for i in range(U.shape[0]):
    js = []
    for j in range(V.shape[0]):
        r = (B[i] + C[j] + mu - A[i, j]) * V[j]
        js.append(r)
    right.append(np.array(js).sum(axis=0))
right = np.array(right)

In [207]:
# def non_u_component(U, B, C, mu, A):
#     new_vec = []
#     for i in range(U.shape[0]):
#         s = (B[i] + C + mu - A[i, :])
#         new_vec.append(s)
#         nuc = np.array(new_vec)
#     return nuc * V

In [166]:
# def non_v_component(U, B, C, mu, A):
#     new_vec = []
#     for i in range(V.shape[0]):
#         s = (B[i] + C + mu - A[i, :]) * V
#         new_vec.append(s)
#         nc = np.array(new_vec)
#     return nc